# Compare Turbine Centerline Wakes for Unstable ABL

In [1]:
# Load the libraries                                                                                                                        

import numpy             as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
#import xarray as xr
#import argparse
import os
import pandas as pd
from netCDF4 import Dataset
import glob

import yaml as yaml
Loader=yaml.safe_load

# Make all plots inline 
%matplotlib inline

In [2]:
def loadyamldef(yamlfile, extradict={}):
    with open(yamlfile) as f:
        returndict = Loader(f)
    # Get the path to yamlfile
    dirpath = os.path.dirname(yamlfile)
    returndict['dirpath'] = dirpath
    # override any entries in the file
    for k,g in extradict.items():
        returndict[k] = g
    return returndict

def getturblist(yamldef, group):
    """
    Return a list of all turbine names in group
    """
    # First check to make sure that the group exists:
    if group not in yamldef:
        return None 
    turbinegroup = yamldef[group]
    turblist = [x['name'] for x in turbinegroup]
    return turblist

def avgturbfiles(yamldef, group, turblist, wakelocations):
    """
    Average turbine wakes from a bunch of turbines
    """
    # First check to make sure that the group exists:
    if group not in yamldef:
        return None 
    turbinegroup = yamldef[group]
    skiprows = yamldef['skiprows'] if 'skiprows' in yamldef else 0
    avgwake = None
    Uh = None
    icount = 1
    for turb in turbinegroup:
        if turb['name'] in turblist:
            turbfile = os.path.join(yamldef['dirpath'], turb[wakelocations])
            dat = np.loadtxt(turbfile, skiprows=skiprows)
            if avgwake is None:
                avgwake  = dat
                Uh       = np.sqrt(dat[:,1]**2 + dat[:,2]**2)
            elif dat.shape == avgwake.shape:
                avgwake += dat
                Uh       += np.sqrt(dat[:,1]**2 + dat[:,2]**2)
                icount += 1
    avgwake /= float(icount)
    Uh /= float(icount)
    return avgwake, Uh

In [3]:
# Edit this list to include data
#           # YAML file,                                                dictionary with extra labels and stuff
datasets = [
    ['DATA_Summit_amrwind_bananasrun1/KingPlainsCenterlines.yaml', {'label':'AMR-Wind Summit', 'plotargs':{'color':'r'}}],
    ['DATA_FLORIS_GCH_unstable/KingPlainsWakes.yaml',              {'label':'FLORIS GCH', 'plotargs':{'color':'g'}}],
    ['DATA_SNL_naluwind_unstable_KPeast/KingPlainsCenterlines_NaluWind.yaml', {'label':'Nalu-Wind SNL', 'plotargs':{'color':'b'}}],
]

In [4]:
datadefs = []
for data in datasets:
    datadefs.append(loadyamldef(data[0], extradict=data[1]))

Loading AMR-Wind Summit
Loading FLORIS GCH
Loading Nalu-Wind SNL


Text(0.5, 1.0, 'King Planes turbine centerline, avg over all turbines')

In [5]:
# Plot overall power
fig, ax = plt.subplots(1,1,figsize=(8,3), dpi=125)
for data in datadefs:
    print('Loading '+data['label'])
    plotargs = data['plotargs'] if 'plotargs' in data else {}
    plotargs['label'] = data['label']
    allturbs=getturblist(data, 'turbinerun')
    wturbs,  avg_Uh_wturbs  = avgturbfiles(data, 'turbinerun',   allturbs, 'centerline')
    noturbs, avg_Uh_noturbs = avgturbfiles(data, 'precursorrun', allturbs, 'centerline')
    x=wturbs[:,0]/data['rotorD']
    ax.plot(x, avg_Uh_wturbs/avg_Uh_noturbs, **plotargs)
    #ax.plot(x, avg_Uh_wturbs/avg_Uh_noturbs, label=data['label'])
    
ax.legend()
ax.grid(ls=':')
ax.set_xlabel('x/D')
ax.set_ylabel('$U_h$ (turbines)/$U_h$ (no turbines)')
ax.set_ylim([0.4, 1.05])
ax.set_title('King Planes turbine centerline, avg over all turbines')